In [4]:
import os
import json
import pandas as pd

In [5]:
business_name="UNI_PRESIDENT_ENTERPRISES"

In [6]:
csv_path = f"result_global_4omini_{business_name}.csv"
df_result = pd.read_csv(csv_path)
df_result["retrieved_contexts"] = df_result["retrieved_contexts"].apply(json.loads)
df_result.loc[20]

FileNotFoundError: [Errno 2] No such file or directory: 'result_global_4omini_UNI_PRESIDENT_ENTERPRISES.csv'

In [ ]:
csv_path = f"result_local_4omini_{business_name}.csv"
df_result = pd.read_csv(csv_path)
df_result["retrieved_contexts"] = df_result["retrieved_contexts"].apply(json.loads)
df_result.loc[20]

user_input                   mean
retrieved_contexts             []
response                     mean
reference                    mean
context_recall           0.754222
context_entity_recall    0.030127
answer_relevancy         0.654929
faithfulness             0.924065
factual_correctness        0.4805
semantic_similarity      0.826613
Name: 20, dtype: object

In [ ]:
csv_path = f"result_rag_{business_name}.csv"
df_result = pd.read_csv(csv_path)
df_result["retrieved_contexts"] = df_result["retrieved_contexts"].apply(json.loads)
df_result.loc[20]

user_input                   mean
retrieved_contexts             []
response                     mean
reference                    mean
context_recall            0.83488
context_entity_recall    0.150757
answer_relevancy         0.699264
faithfulness             0.926176
factual_correctness         0.519
semantic_similarity      0.812671
Name: 20, dtype: object

In [29]:
import pandas as pd
import json

# 模擬 business_name
business_name = "USI"

# 定義 CSV 路徑
csv_paths = [
    f"./show/result_global_4omini_{business_name}.csv",
    f"./show/result_local_4omini_{business_name}.csv",
    f"./show/result_community_4omini_{business_name}.csv",
    f"./show/result_rag_{business_name}.csv"
]

# 定義結果的列標題
columns = ["metrics", "global_4omini", "local_4omini", "community_4omini","rag"]

# 初始化空的 DataFrame
result_df = pd.DataFrame(columns=columns)

# 載入每個 CSV 並提取對應的數值
data = {}
metric_names = None

for idx, csv_path in enumerate(csv_paths):
    # 讀取 CSV
    df_result = pd.read_csv(csv_path)
    # 確保 retrieved_contexts 被正確載入
    df_result["retrieved_contexts"] = df_result["retrieved_contexts"].apply(json.loads)
    
    # 提取第 20 行作為數據來源
    row = df_result.loc[20]

    # 儲存對應欄位的數據
    if idx == 0:
        # 初始化 metrics（只需要一次）
        metric_names = [
            "context_recall",
            "context_entity_recall",
            "answer_relevancy",
            "faithfulness",
            "factual_correctness",
            "semantic_similarity"
        ]
        data["metrics"] = metric_names

    # 提取當前 CSV 的數據
    data[csv_path.split("_")[1]] = [row[metric] for metric in metric_names]

# 將數據轉為 DataFrame
result_df = pd.DataFrame(data)
result_df.columns = ["metrics", "global_4omini", "local_4omini", "community_4omini", "rag"]

# 將數值格式化為小數點後兩位
result_df[["global_4omini", "local_4omini", "community_4omini", "rag"]] = result_df[["global_4omini", "local_4omini", "community_4omini", "rag"]].round(2)

# 顯示結果
result_df


,metrics,global_4omini,local_4omini,community_4omini,rag
0,context_recall,0.54,0.75,0.76,0.83
1,context_entity_recall,0.10,0.03,0.09,0.15
2,answer_relevancy,0.59,0.65,0.64,0.70
3,faithfulness,0.92,0.92,0.85,0.93
4,factual_correctness,0.37,0.48,0.43,0.52
5,semantic_similarity,0.80,0.83,0.78,0.81


In [30]:
result_df.to_csv(f"./all/result_{business_name}.csv", index=False)

In [34]:
import pandas as pd
import glob

# 指定所有 CSV 檔案的路徑 (用 glob 匹配所有 CSV 檔案)
file_paths = glob.glob("./all/*.csv")  # 替換為您的檔案路徑

# 初始化空的 DataFrame 用於儲存平均值
average_df = None

# 讀取所有 CSV 檔案並累加
for file_path in file_paths:
    # 讀取 CSV 檔案
    df = pd.read_csv(file_path)
    
    # 如果是第一個檔案，初始化 average_df
    if average_df is None:
        average_df = df.copy()
    else:
        # 將數值列累加
        for col in df.columns[1:]:  # 跳過第一列 (metrics)
            average_df[col] += df[col]

# 將累加值除以檔案數量以獲得平均值
average_df.iloc[:, 1:] = average_df.iloc[:, 1:] / len(file_paths)

average_df[["global_4omini", "local_4omini", "community_4omini", "rag"]] = average_df[["global_4omini", "local_4omini", "community_4omini", "rag"]].round(2)

# 顯示結果
average_df


,metrics,global_4omini,local_4omini,community_4omini,rag
0,context_recall,0.58,0.72,0.78,0.81
1,context_entity_recall,0.12,0.06,0.11,0.16
2,answer_relevancy,0.46,0.62,0.47,0.59
3,faithfulness,0.89,0.83,0.87,0.90
4,factual_correctness,0.38,0.44,0.38,0.45
5,semantic_similarity,0.81,0.79,0.75,0.77


In [ ]:
# 儲存結果到新的 CSV 檔案
average_df.to_csv("result.csv", index=False)

In [2]:
import pandas as pd
import glob
from scipy.stats import pearsonr

metric = "semantic_similarity"

# 指定所有 CSV 檔案的路徑 (用 glob 匹配所有 CSV 檔案)
file_paths = glob.glob("all/*.csv")  # 替換為您的檔案路徑

# 初始化列表存儲 context_recall 的值
context_recall_values = []

# 從每個 CSV 檔案中提取 context_recall 的數據
for file_path in file_paths:
    # 讀取 CSV 檔案
    df = pd.read_csv(file_path)
    
    # 提取 context_recall 的數值 (假設 metrics 是第一欄)
    context_recall_row = df[df["metrics"] == metric]
    
    # 將數值追加到列表中
    context_recall_values.append(context_recall_row.iloc[0, 1:].values)  # 跳過第一欄（metrics）

# 將數據轉為 DataFrame 以便進行統計計算
context_recall_df = pd.DataFrame(context_recall_values, columns=["global_4omini", "local_4omini", "community_4omini", "rag"])

# 計算統計值
stats = {
    "metrics": ["mean", "Q1", "median", "Q3"],
    "global_4omini": [
        context_recall_df["global_4omini"].mean(),
        context_recall_df["global_4omini"].quantile(0.25),  # Q1
        context_recall_df["global_4omini"].median(),
        context_recall_df["global_4omini"].quantile(0.75),  # Q3
    ],
    "local_4omini": [
        context_recall_df["local_4omini"].mean(),
        context_recall_df["local_4omini"].quantile(0.25),
        context_recall_df["local_4omini"].median(),
        context_recall_df["local_4omini"].quantile(0.75),
    ],
    "community_4omini": [
        context_recall_df["community_4omini"].mean(),
        context_recall_df["community_4omini"].quantile(0.25),
        context_recall_df["community_4omini"].median(),
        context_recall_df["community_4omini"].quantile(0.75),
    ],
    "rag": [
        context_recall_df["rag"].mean(),
        context_recall_df["rag"].quantile(0.25),
        context_recall_df["rag"].median(),
        context_recall_df["rag"].quantile(0.75),
    ],
}

# 將統計值轉為 DataFrame
stats_df = pd.DataFrame(stats)

# 計算 p-value
p_values = {
    "metrics": ["p-value"],
    "global_4omini": [pearsonr(context_recall_df["global_4omini"], context_recall_df["rag"])[1]],
    "local_4omini": [pearsonr(context_recall_df["local_4omini"], context_recall_df["rag"])[1]],
    "community_4omini": [pearsonr(context_recall_df["community_4omini"], context_recall_df["rag"])[1]],
    "rag": ["N/A"]  # 因為 rag 與自己無法計算 p-value
}

# 將 p-value 轉為 DataFrame
p_values_df = pd.DataFrame(p_values)

p_values_df[["global_4omini", "local_4omini", "community_4omini", "rag"]] = p_values_df[["global_4omini", "local_4omini", "community_4omini", "rag"]].round(2)

# 顯示結果
p_values_df

,metrics,global_4omini,local_4omini,community_4omini,rag
0,p-value,0.0,0.0,0.0,N/A


In [ ]:
stats_df

In [16]:
# 儲存結果到新的 CSV 檔案
stats_df.to_csv(f"./result_{metric}.csv", index=False)

In [12]:
df_show = pd.read_csv("result_context_recall.csv")
df_show

,metrics,global_4omini,local_4omini,community_4omini,rag
0,mean,0.58,0.72,0.78,0.81
1,Q1,0.50,0.64,0.76,0.76
2,median,0.57,0.73,0.78,0.82
3,Q3,0.65,0.81,0.82,0.84
